<a href="https://colab.research.google.com/github/pbhuch/GL_DecA_G4_NLP1/blob/NLP1_PM/Capstone_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Project Modelling part**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install ftfy

     |████████████████████████████████| 64 kB 2.1 MB/s 
  Created wheel for ftfy: filename=ftfy-6.0.3-py3-none-any.whl size=41933 sha256=98046702122ce7690e31551b5e964402626032268f1914052e7924577ef547ab
  Stored in directory: /root/.cache/pip/wheels/19/f5/38/273eb3b5e76dfd850619312f693716ac4518b498f5ffb6f56d
Successfully built ftfy


In [125]:
# importing libraries
from time import time
from PIL import Image
from zipfile import ZipFile
import os, sys, itertools, re
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
import plotly as py
import plotly.graph_objs as go
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot, plot
from sklearn.preprocessing import QuantileTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score, confusion_matrix, classification_report
import sklearn.neighbors._base
#import sys
#sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
import imblearn
from imblearn.over_sampling import SMOTE

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, Activation, Conv2D, MaxPooling2D, Reshape, Embedding, LSTM,  TimeDistributed, Bidirectional, Lambda, Input, Add, GlobalMaxPool1D
from tensorflow.keras import regularizers, optimizers
from sklearn.metrics import r2_score
from tensorflow.keras.models import load_model
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
#import cv2
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# to define loss
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.backend import log, epsilon

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.datasets import imdb

from itertools import islice

import re
import nltk
from nltk.corpus import stopwords

from ftfy import fix_encoding, fix_text, badness
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

from sklearn.utils import resample

import pickle, string

import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

import plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot

import spacy

# Suppress warnings
import warnings; warnings.filterwarnings('ignore')

SEED = 123                 # to be able to rerun the same NN
np.random.seed(SEED)
tf.random.set_seed(SEED)

from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = None




import time
# ML Modeling
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score, confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

In [126]:
import os; 
os.chdir('/content/drive/MyDrive/Great_Learning/Capstone')

In [127]:
dataset = pd.read_csv("dataset1_mojibaked_LangDetect_NewColumnAdded.csv", encoding = 'utf-8')

In [128]:
dataset.head()

,Unnamed: 0,Short description,Description,Caller,Assignment group,combined_description_caller,combined_description,ConvertedToEnglish,Language
0,0,login issue,verified user detailsemployee manager name che...,spxjnwir pjlcoqds,GRP_0,login issue verified user detailsemployee mana...,login issue verified user detailsemployee mana...,NaN,en
1,1,outlook,received from hmjdrvpb.komuaywn@gmail.com\n he...,hmjdrvpb komuaywn,GRP_0,outlook received from hmjdrvpb.komuaywn@gmail....,outlook received from hmjdrvpb.komuaywn@gmail....,NaN,en
2,2,cant log in to vpn,received from eylqgodm.ybqkwiam@gmail.com\n hi...,eylqgodm ybqkwiam,GRP_0,cant log in to vpn received from eylqgodm.ybqk...,cant log in to vpn received from eylqgodm.ybqk...,NaN,en
3,3,unable to access hrtool page,unable to access hrtool page,xbkucsvz gcpydteq,GRP_0,unable to access hrtool page unable to access ...,unable to access hrtool page unable to access ...,NaN,en
4,4,skype error,skype error,owlgqjme qhcozdfx,GRP_0,skype error skype error owlgqjme qhcozdfx,skype error skype error,NaN,no


In [129]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8417 entries, 0 to 8416
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   8417 non-null   int64  
 1   Short description            8408 non-null   object 
 2   Description                  8413 non-null   object 
 3   Caller                       8417 non-null   object 
 4   Assignment group             8417 non-null   object 
 5   combined_description_caller  8417 non-null   object 
 6   combined_description         8417 non-null   object 
 7   ConvertedToEnglish           0 non-null      float64
 8   Language                     8417 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 591.9+ KB


In [ ]:
dataset['Assignment group'].value_counts()


GRP_0     3934
GRP_8      645
GRP_24     285
GRP_12     257
GRP_9      252
GRP_2      241
GRP_19     215
GRP_3      200
GRP_6      183
GRP_13     145
GRP_10     140
GRP_5      128
GRP_14     118
GRP_25     116
GRP_33     107
GRP_4      100
GRP_29      97
GRP_18      88
GRP_16      85
GRP_31      69
GRP_17      68
GRP_7       68
GRP_34      62
GRP_26      56
GRP_40      45
GRP_28      44
GRP_41      40
GRP_30      39
GRP_15      38
GRP_42      37
GRP_20      36
GRP_45      35
GRP_22      31
GRP_1       31
GRP_11      30
GRP_21      28
GRP_47      27
GRP_48      25
GRP_62      25
GRP_23      25
GRP_39      19
GRP_27      18
GRP_60      16
GRP_37      16
GRP_36      15
GRP_44      15
GRP_50      14
GRP_65      11
GRP_53      11
GRP_52       9
GRP_55       8
GRP_51       8
GRP_46       6
GRP_59       6
GRP_49       6
GRP_43       5
GRP_66       4
GRP_32       4
GRP_68       3
GRP_58       3
GRP_63       3
GRP_56       3
GRP_38       3
GRP_71       2
GRP_57       2
GRP_72       2
GRP_54    

In [130]:
dataset['Language'].value_counts()

en       7010
de        415
af        271
it        121
fr        105
no         71
nl         68
sv         53
zh-cn      49
es         47
ca         39
pl         27
da         25
ko         17
pt         17
tl         11
cy         11
ro         10
et         10
sq          8
sl          6
hr          6
fi          5
vi          3
id          3
so          2
lt          2
cs          2
ja          1
tr          1
hu          1
Name: Language, dtype: int64

In [131]:
data_eng = dataset[dataset['Language']=='en']

In [132]:
data_eng.head()

,Unnamed: 0,Short description,Description,Caller,Assignment group,combined_description_caller,combined_description,ConvertedToEnglish,Language
0,0,login issue,verified user detailsemployee manager name che...,spxjnwir pjlcoqds,GRP_0,login issue verified user detailsemployee mana...,login issue verified user detailsemployee mana...,NaN,en
1,1,outlook,received from hmjdrvpb.komuaywn@gmail.com\n he...,hmjdrvpb komuaywn,GRP_0,outlook received from hmjdrvpb.komuaywn@gmail....,outlook received from hmjdrvpb.komuaywn@gmail....,NaN,en
2,2,cant log in to vpn,received from eylqgodm.ybqkwiam@gmail.com\n hi...,eylqgodm ybqkwiam,GRP_0,cant log in to vpn received from eylqgodm.ybqk...,cant log in to vpn received from eylqgodm.ybqk...,NaN,en
3,3,unable to access hrtool page,unable to access hrtool page,xbkucsvz gcpydteq,GRP_0,unable to access hrtool page unable to access ...,unable to access hrtool page unable to access ...,NaN,en
5,5,unable to log in to engineering tool and skype,unable to log in to engineering tool and skype,eflahbxn ltdgrvkz,GRP_0,unable to log in to engineering tool and skype...,unable to log in to engineering tool and skype...,NaN,en


In [133]:
# Create a target categorical column
data_eng['target'] = data_eng['Assignment group'].astype('category').cat.codes
data_eng.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7010 entries, 0 to 8415
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   7010 non-null   int64  
 1   Short description            7002 non-null   object 
 2   Description                  7006 non-null   object 
 3   Caller                       7010 non-null   object 
 4   Assignment group             7010 non-null   object 
 5   combined_description_caller  7010 non-null   object 
 6   combined_description         7010 non-null   object 
 7   ConvertedToEnglish           0 non-null      float64
 8   Language                     7010 non-null   object 
 9   target                       7010 non-null   int8   
dtypes: float64(1), int64(1), int8(1), object(7)
memory usage: 554.5+ KB


In [134]:
data_eng['target'].value_counts()

0     3138
72     642
73     249
12     235
4      224
11     199
23     194
56     180
5      142
2      134
45     126
6      115
34     100
18      97
22      94
10      87
8       83
67      68
9       65
17      59
19      53
28      47
35      42
36      40
7       36
40      34
13      34
21      34
25      33
15      31
3       29
1       29
14      28
27      28
42      27
16      24
33      19
57      16
31      16
20      16
39      14
46      13
30      13
62      11
49      11
59      10
48       8
51       8
24       8
47       8
38       5
55       5
37       4
63       4
41       4
60       3
44       3
65       3
32       3
54       3
52       3
69       2
53       2
26       2
50       2
66       2
43       2
64       1
68       1
29       1
61       1
58       1
70       1
71       1
Name: target, dtype: int64

In [163]:
# Create training and test datasets with 80:20 ratio
X_train, X_test, y_train, y_test = train_test_split(data_eng.combined_description, 
                                                    data_eng.target, 
                                                    test_size=0.20, 
                                                    random_state=42)
print('\033[1mShape of the training set:\033[0m', X_train.shape, X_test.shape)
print('\033[1mShape of the test set:\033[0m', y_train.shape, y_test.shape)

Shape of the training set: (5608,) (1402,)
Shape of the test set: (5608,) (1402,)


**MACHINE LEARNING**

In [172]:
def fit_n_print(model, X_train, X_test, y_train, y_test):  # take the model, train data and test data as input
    
    start = time.time()  # note the start time 

    clf = model
    clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('model', model),
                     ])

    clf.fit(X_train, y_train)   # fit the model using the train data

    pred_train=clf.predict(X_train) # model predictions on the training data
    y_pred = clf.predict(X_test)   # model predictions on the test data
    
    accuracy_training = (accuracy_score(y_train,pred_train))*100 #accurancy on training
    accuracy_test= (accuracy_score(y_test,y_pred ))*100  #accuracy on test
    

    recallscore_training =(recall_score(y_train,pred_train,average='macro'))*100  #recall on training
    recallscore_test = (recall_score(y_test,y_pred, average='macro'))*100  #recall on test
    
    precision_training = (precision_score(y_train,pred_train, average='macro'))*100
    precision_test = (precision_score(y_test,y_pred,average='macro'))*100
    
    f1score_training = (f1_score(y_train,pred_train, average='macro'))*100
    f1score_test = (f1_score(y_test,y_pred, average='macro'))*100
        
    end = time.time()  #note the end time
       
    duration = end - start  # calculate the total duration

    print('Algorithm:', type(model).__name__)
    print("\n Classification report:\n", classification_report(y_test, y_pred))
    print("\n Confusion report:\n", confusion_matrix(y_test, y_pred))
    print("Accuracy Score:", accuracy_score(y_test, y_pred))
    print()
    print("\n \n")
     
    

    return  accuracy_training,accuracy_test,recallscore_training, recallscore_test, precision_training,precision_test,f1score_training, f1score_test,  duration, y_pred  # return all the metrics along with predictions





In [173]:
rf  = RandomForestClassifier()
xgb = XGBClassifier()
SVC = LinearSVC()
KNN = KNeighborsClassifier()
NB = MultinomialNB()

result = {}   # Create an empty dictionary to later use to store metrics of each of the models

for model, name  in zip([rf,xgb, SVC,KNN,NB], 
                         ['Random Forest', 'Xgboost', 'SVC','KNN','Naive Bayes']):
    result[name] = fit_n_print(model,X_train, X_test, y_train, y_test)

Algorithm: RandomForestClassifier

 Classification report:
               precision    recall  f1-score   support

           0       0.60      1.00      0.75       629
           1       0.00      0.00      0.00         8
           2       0.92      0.31      0.46        36
           3       0.00      0.00      0.00         6
           4       0.50      0.27      0.35        41
           5       0.43      0.14      0.21        21
           6       0.77      0.36      0.49        28
           7       0.00      0.00      0.00        11
           8       0.00      0.00      0.00        13
           9       1.00      0.94      0.97        16
          10       0.00      0.00      0.00        15
          11       0.50      0.06      0.10        36
          12       1.00      0.21      0.34        48
          13       0.00      0.00      0.00        10
          14       0.00      0.00      0.00         5
          15       0.00      0.00      0.00         7
          16       0.

In [174]:
result_normal = pd.DataFrame(np.array(list(result.values()))[:,:-1],    # make a dataframe out of the metrics from result dictionary 
                       columns= ['accuracy_training','accuracy_test',
                                 'recallscore_training', 'recallscore_test', 
                                 'precision_training','precision_test',
                                 'f1score_training', 'f1score_test', 
                                 'Elapsed'],
                      index= result.keys())   # use the model names as index

result_normal.index.name = 'Model'   # name the index of the result1 dataframe as 'Model'

result_normal

,accuracy_training,accuracy_test,recallscore_training,recallscore_test,precision_training,precision_test,f1score_training,f1score_test,Elapsed
Model,,,,,,,,,
Random Forest,94.7218,61.9829,93.9805,13.1849,97.1229,24.9512,95.241,15.3755,11.1908
Xgboost,80.2782,62.1969,68.3495,19.4961,87.3594,28.2875,74.7527,21.4228,164.807
SVC,92.9565,67.9743,90.9303,32.215,97.0378,43.1268,93.2581,34.9384,1.21744
KNN,68.3845,63.1241,24.1808,19.4539,52.0372,34.0514,29.9675,22.6188,2.31285
Naive Bayes,54.2083,53.923,3.05674,3.42775,5.79664,2.37972,2.54894,2.58779,0.604044


**DEEP LEARNING**

In [139]:
gloveFileName = '/content/drive/MyDrive/Great_Learning/NLP_sarcasm_detection/glove.6B/glove.6B.200d.txt'

In [140]:
#Tokenizing and Padding the Independent features

from tensorflow.python.keras.preprocessing.sequence import pad_sequences

MAX_LENGTH = 300
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data_eng.combined_description.values)
post_seq = tokenizer.texts_to_sequences(data_eng.combined_description.values)
post_seq_padded = pad_sequences(post_seq, maxlen=MAX_LENGTH)

In [141]:
y = data_eng['target']

In [142]:
#Splitting the data into Train and Test (Ratio = 0.20)
X_train, X_test, y_train, y_test = train_test_split(post_seq_padded, y, test_size=0.2, random_state=42)

In [143]:
X_train.shape,X_test.shape

((5608, 300), (1402, 300))

In [144]:
y_train.shape, y_test.shape

((5608,), (1402,))

In [145]:
#Creating input dimension ('vocab_size')

vocab_size = len(tokenizer.word_index) + 1
max_features = vocab_size 
vocab_size

15541

In [146]:
# Creating Output dimension ('num_class')
num_class = len(np.unique(data_eng['target'].values))
num_class

74

**Implementing Bi LSTM**

In [147]:
max_features = vocab_size  
maxlen = 200
seq_output_size = 128
filters = 64
kernel_size = 5
pool_size = 4
activation_func = 'sigmoid'

In [148]:
from keras import layers
# For Embedding Layer
from keras.layers.embeddings import Embedding


# Bi directional LSTM
# Input for variable-length sequences of integers
inputs = keras.Input(shape=(None,), dtype="int32")
# Embed each integer in a 128-dimensional vector
x = layers.Embedding(max_features, 128)(inputs)
x = layers.Dropout(0.2)(x)
# Add 2 bidirectional LSTMs
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(32, activation='relu')(x)
outputs = layers.Dense(num_class, activation='softmax')(x)
# Add a classifier
bilstm_model = keras.Model(inputs, outputs)
bilstm_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 128)         1989248   
                                                                 
 dropout (Dropout)           (None, None, 128)         0         
                                                                 
 bidirectional (Bidirectiona  (None, None, 128)        98816     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 128)               0     

In [149]:
#model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
bilstm_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])
bilstm_model.fit(X_train, y_train, batch_size=20, epochs=7, validation_data=(X_test, y_test))

Epoch 1/7
281/281 [==============================] - 224s 765ms/step - loss: 2.4381 - acc: 0.5018 - val_loss: 2.0595 - val_acc: 0.5421
Epoch 2/7
281/281 [==============================] - 214s 760ms/step - loss: 1.9049 - acc: 0.5446 - val_loss: 1.9983 - val_acc: 0.5492
Epoch 3/7
281/281 [==============================] - 214s 762ms/step - loss: 1.7174 - acc: 0.5651 - val_loss: 2.0902 - val_acc: 0.5414
Epoch 4/7
281/281 [==============================] - 214s 762ms/step - loss: 1.5482 - acc: 0.5952 - val_loss: 2.1155 - val_acc: 0.5121
Epoch 5/7
281/281 [==============================] - 214s 762ms/step - loss: 1.3914 - acc: 0.6289 - val_loss: 2.1216 - val_acc: 0.4765
Epoch 6/7
281/281 [==============================] - 212s 755ms/step - loss: 1.2440 - acc: 0.6523 - val_loss: 2.1695 - val_acc: 0.5307
Epoch 7/7
281/281 [==============================] - 212s 754ms/step - loss: 1.1261 - acc: 0.6744 - val_loss: 2.2275 - val_acc: 0.5178


In [150]:
bi_lstm_predicted = bilstm_model.predict(X_test)
bi_lstm_predicted = np.argmax(bi_lstm_predicted, axis=1)
accuracy_score(y_test, bi_lstm_predicted)

0.5178316690442225

**Machine Learning Models.**
**bold text**
5 Machine learning models – Random Forests, XGBoost, SVC, kNN, Naïve Bayes were selected.
All models help in classification problems. Hence, the selection

**Random Forests**
Random forests or random decision forests are an ensemble learning method for classification, regression.
Algorithm creates multiple of decision trees at training time and output of the class is the mode of the classes (classification) or mean prediction (regression) of the individual trees. 
Random decision forests correct for decision trees’ habit of overfitting to their training set. 
In two-dimensional space this hyperplane is a line dividing a plane in two parts where in each class lay in either side.

**XGBoost**
XGBoost stands for eXtreme Gradient Boosting.
XGBoost is an ensemble learning algorithm meaning that it combines the results of many models, called base learners to make a prediction. Like Random Forests, XGBoost uses Decision Trees as base learners.
Individual decision trees are low-bias, high-variance models. They are incredibly good at finding the relationships in any type of training data but struggle to generalize well on unseen data.
XGBoost is an implementation of gradient boosted decision trees designed for speed and performance. 
Its speed and performance are unparalleled and it consistently outperforms any other algorithms aimed at supervised learning tasks. 

**Linear SVC**
The Support Vector Machine (SVM) technique is a popular and highly accurate machine learning method for classification problems. SVM try to find an optimal hyperplane within the input space to correctly classify the binary (or multi-class)
classification problem. In the two-dimensional space this hyperplane is a line dividing a plane in two parts where in each class lay in either side. 

**KNN**
K Nearest Neighbors is a  Supervised Learning algorithm and is  mostly used for and also regression. 
KNN is a non-parametric and lazy learning algorithm. 
Non-parametric means there is no assumption for underlying data distribution. 
Lazy algorithms do not need any training data points for model generation. 
All training data used in the testing phase. 
Hence. training faster and testing phase slower and costlier leading to more memory usage and also more time.
KNN algorithm is uses distances such as Euclidean, Manhattan and Minkowski to find the K nearest neighbors in the training data and then uses these labels to predict


**Naïve Bayes**
A Naive Bayes classifier is an algorithm that uses Bayes’ theorem to classify objects. Naive Bayes classifiers assume strong, or naive, independence between attributes of data points. The key insight of Bayes’ theorem is that the probability of an event can be adjusted as new data is introduced. These classifiers are widely used for machine learning because they are simple to implement.

**Deep learning models**

**Bi-LSTM:(Bi-directional long short term memory):**
Bidirectional recurrent neural networks(RNN) are really just putting two independent RNNs together. This structure allows the networks to have both backward and forward information about the sequence at every time step
Using bidirectional will run the inputs in two ways, one from past to future and one from future to past and what differs this approach from unidirectional is that in the LSTM that runs backward you preserve information from the future and using the two hidden states combined you are able in any point in time to preserve information from both past and future.
As in NLP , sometimes to understand a word we need not just to the previous word , but also to the coming word , like in this example
